In [2]:
import os
import librosa   #thư viện xủ lý tín hiệu âm thanh
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile #để đọc file wav 
import warnings
import tensorflow as tf

In [3]:
# MLP for Pima Indians Dataset Serialize to JSON and HDF5
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model.h5")
model.summary()

Instructions for updating:
Colocations handled automatically by placer.


Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 20, 8, 16)         160       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 20, 8, 16)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 4, 16)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 4, 32)         4640      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 10, 4, 32)         0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 2, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 2, 64)         

In [14]:
import pyaudio
import wave
import struct
CHUNK = 16000 # read each 1000 miliseconds
FORMAT = pyaudio.paFloat32
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 20
WORD = 8000
WORD_HALF = int(WORD/2)
WORD_QUAD = int(WORD/2)
# circular_buf_size = 50
p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)
frames = []
features = []
waves = []
words = []
circular_index = 0
print("* recording")
inputDataNew = np.ndarray(shape=(CHUNK), dtype=float)
inputDataOld = np.ndarray(shape=(WORD_QUAD), dtype=float)
last_word_time = 0
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK) 
    #convert to float array
    for j in range(0, int(len(data)/4)):
        value = struct.unpack('f', data[j*4:j*4+4])
        inputDataNew[j] = value[0]
    #megre end of old and new
    inputData = np.concatenate((inputDataOld,inputDataNew))
    inputDataOld = inputDataNew[CHUNK-WORD_QUAD:].copy()
    if i<1:
        continue
    else:
        mean = np.mean(inputData)
        energy = abs(inputData)
        value = 0
        max_val = 0
        max_idn = 0
        for idn in range(0,len(energy)):
            value += (energy[idn]-value)/300
            if (idn>WORD_HALF) & (idn<(len(inputData)-WORD_HALF )):
                if max_val<value:
                    max_val = value
                    max_idn = idn
        if max_val>(mean*3):
            global_time = i*CHUNK+max_idn-WORD_QUAD
            if (global_time-last_word_time)< WORD:
                continue
            last_word_time = global_time
            data = inputData[max_idn-WORD_HALF:max_idn+WORD_HALF]
            feature_vector = librosa.feature.melspectrogram(y=data, sr=16000)[:40]
            modelInput = np.array(feature_vector).reshape(1,feature_vector.shape[0],feature_vector.shape[1],1)
            output = model.predict(modelInput)
            indexMax = np.argmax(output[0], axis=0) 
            if (indexMax!=10) :
#             os.system( 'cls' )
                print ("ID:",len(waves)," prediction:",indexMax," time:",global_time/RATE)
                waves.append(inputData)
                words.append(data)
                features.append(feature_vector)
#             plt.figure(figsize=(10, 4))
#             librosa.display.specshow(chroma, y_axis='chroma', x_axis='time')
#             plt.colorbar()
#             plt.title('Chromagram')
#             plt.tight_layout()
#             plt.show()
print("* done recording")    

    

* recording
ID: 0  prediction: 1  time: 2.00325
ID: 1  prediction: 9  time: 3.69425
ID: 2  prediction: 6  time: 5.13325
ID: 3  prediction: 9  time: 7.0001875
ID: 4  prediction: 1  time: 8.0000625
ID: 5  prediction: 6  time: 9.0815
ID: 6  prediction: 6  time: 10.6623125
ID: 7  prediction: 6  time: 11.579875
ID: 8  prediction: 6  time: 12.690875
ID: 9  prediction: 3  time: 13.681375
* done recording


In [12]:
testId =0
modelInput = np.array(features[testId]).reshape(1,feature_vector.shape[0],feature_vector.shape[1],1)
output = model.predict(modelInput)
indexMax = np.argmax(output[0], axis=0) 
print(indexMax)
print(output)
ipd.Audio(words[testId], rate=16000)

1
[[3.9031318e-01 6.0968643e-01 1.3507352e-08 7.9962946e-13 6.3308975e-10
  7.9423412e-11 1.1263025e-22 7.0499669e-16 1.4850192e-33 3.5079316e-07
  1.7982466e-16]]


In [9]:
len(waves[testId])

20000

In [36]:
import random
for i in range(0,5):
    filename = 'D:/DATA/voice/train/audio/1/retrain'+str(random.randrange(10000))+'.wav'
    y= librosa.resample(waves[testId], 16000, 22050)
    librosa.output.write_wav(filename, y, 22050)
    print(filename)

D:/DATA/voice/train/audio/1/retrain5502.wav
D:/DATA/voice/train/audio/1/retrain5275.wav
D:/DATA/voice/train/audio/1/retrain8777.wav
D:/DATA/voice/train/audio/1/retrain2276.wav
D:/DATA/voice/train/audio/1/retrain6659.wav


In [16]:
modelInput = np.array(features).reshape(len(features),12,43,1)
output = model.predict(modelInput)
for i in range(0, len(output)):
    indexMax = np.argmax(output[i], axis=0) 

ValueError: cannot reshape array of size 5760 into shape (9,12,43,1)

In [ ]:
import matplotlib.pyplot as plt
import librosa.display

plt.figure(figsize=(10, 4))
librosa.display.specshow(features[5], y_axis='chroma', x_axis='time')
plt.colorbar()
plt.title('Chromagram')
plt.tight_layout()
plt.show()

In [ ]:
len(inputData)

In [ ]:
(chroma.shape)

In [ ]:
samples